This isn't spacy, so we need to do the preprocessing ourselves.

Summarised from: https://www.kaggle.com/theoviel/improve-your-score-with-some-text-preprocessing

In [1]:
import pandas as pd
import numpy as np
import operator 
import re

pd.set_option('display.max_colwidth', -1)

# Loading Questions

In [2]:
df = pd.read_csv("../input/train.csv")

sample of sincere questions

In [3]:
df.loc[df['target'] == 0].sample(n=5)

,qid,question_text,target
321281,3ef813a4e9b48391baa4,Where can I see full length porn videos for free?,0
185869,245b230b89e7526e7cc4,Why will a car at point A and a lorry take different direction and still arrive at the same time?,0
729378,8ed13e1bbe4ab45a66be,What is a tart?,0
1115982,daaf7360600033718b8a,Is There are any secrets to making money in the SGX stock market?,0
279158,36a3cd570a251a99a365,Are you going to ask me a question?,0


sample of insincere questions

In [4]:
df.loc[df['target'] == 1].sample(n=5)

,qid,question_text,target
889471,ae42e5c5f771d669bce9,Who is better: North Indians or South Indians?,1
955537,bb3a39aa1fd4bdd2f5a9,Are Filipinos lazy and therefore poor?,1
416667,51a9223b3be3cd97bfd4,What happens if I put Frosted Flakes in a grilled cheese sandwich? Should I have a tetanus shot ready at hand?,1
788935,9a922c4f24ecd516f3a8,Why has Trump made North Korea Nuclear weapon and missile a great success and ten stopped the military drill to have a talk between North & South Korea? Why has Trump failed miserably in the world?,1
1125555,dc91ee421f7191519e33,"Why do people admire Steve Jobs, a man hated by everyone who met him, who cried like a brat, lied, cheated and stole other people's ideas and whose only singular accomplishment was dying from a cancer most people survive?",1


# Cleaning questions

In [18]:
df['treated_question'] = df['question_text'].apply(lambda x: x.lower())

In [19]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

df['treated_question'] = df['treated_question'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [20]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"’": "'", "‘": "'", "´": "'", "`": "'", "‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    text = re.sub(r'[^\x20-\x7e]',r'', text) 

    return text

df['treated_question'] = df['treated_question'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [21]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

df['treated_question'] = df['treated_question'].apply(lambda x: correct_spelling(x, mispell_dict))

In [22]:
custom_mapping = {'upsc': 'indian government', 'aiims': 'indian medicine', 'quorans': 'quora users', 'brexit': 'eu referendum', 'cryptocurrencies': 'cryptocurrency', 'redmi': 'xiaomi', 'kvpy': 'government scholarship', 'paytm': 'paypal', 'iiser': 'indian institute of science education and research', 'ethereum': 'bitcoin', 'iisc': 'indian institute of science', 'jinping': 'chinese president'}

def clean_contractions(text, mapping):
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

df['treated_question'] = df['treated_question'].apply(lambda x: clean_contractions(x, custom_mapping))

# Check coverage

In [23]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

glove = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
paragram =  '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
wiki_news = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'

# print("Extracting GloVe embedding")
# embed_glove = load_embed(glove)
print("Extracting Paragram embedding")
embed_paragram = load_embed(paragram)
print("Extracting FastText embedding")
embed_fasttext = load_embed(wiki_news)

Extracting Paragram embedding
Extracting FastText embedding


In [24]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

vocab = build_vocab(df['treated_question'])

In [25]:
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.3%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.3%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [26]:
# print("Glove : ")
# oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

Paragram : 
Found embeddings for 74.949% of vocab
Found embeddings for  99.663% of all text
FastText : 
Found embeddings for 48.688% of vocab
Found embeddings for  98.847% of all text


# sampling

In [30]:
# oov_glove[:10]

In [31]:
oov_fasttext[:10]

[('cgl', 930),
 ('jio', 770),
 ('manipal', 741),
 ('icse', 658),
 ('iiit', 564),
 ('bitsat', 562),
 ('cgpa', 559),
 ('ielts', 554),
 ('xiaomi', 526),
 ('mtech', 504)]

In [32]:
oov_paragram[:10]

[('coinbase', 149),
 ('oneplus', 139),
 ('uceed', 123),
 ('bhakts', 115),
 ('upwork', 111),
 ('machedo', 108),
 ('gdpr', 107),
 ('adityanath', 106),
 ('boruto', 102),
 ('bnbr', 100)]